# Notebook C: Multi-Task Disease Classifier
## Training separate heads for TB, Parkinson's, and Pulmonary anomalies

Uses HeAR embeddings to train classifiers for:
- **TB Detection**: Coughvid + Coswara datasets
- **Parkinson's Detection**: Voice dataset
- **Pulmonary Anomalies**: Respiratory Sound Database

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import json
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, roc_auc_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

DATASETS_ROOT = Path(r"D:\datasets")
EMBEDDINGS_DIR = DATASETS_ROOT / 'embeddings'
MODELS_DIR = DATASETS_ROOT / 'models'
MODELS_DIR.mkdir(parents=True, exist_ok=True)

print(f"Embeddings: {EMBEDDINGS_DIR}")
print(f"Models output: {MODELS_DIR}")

Embeddings: D:\datasets\embeddings
Models output: D:\datasets\models


In [2]:
def load_embeddings(dataset_name):
    """Load embeddings for a dataset"""
    path = EMBEDDINGS_DIR / f"{dataset_name}_embeddings.npz"
    if not path.exists():
        print(f"⚠ {dataset_name}: Embeddings not found")
        return None, None
    data = np.load(path, allow_pickle=True)
    return data['embeddings'], data['file_names']

# Load all embeddings
embeddings = {}
file_names = {}
for name in ['coughvid', 'parkinsons', 'respiratory_sounds', 'coswara']:
    emb, files = load_embeddings(name)
    if emb is not None:
        embeddings[name] = emb
        file_names[name] = files
        print(f"✓ {name}: {emb.shape}")

✓ coughvid: (11319, 1024)
✓ parkinsons: (5, 1024)
✓ respiratory_sounds: (9841, 1024)
✓ coswara: (5, 1024)


In [3]:
def load_labels_coughvid():
    """Load labels from Coughvid metadata (COVID status)"""
    csv_path = DATASETS_ROOT / 'coughvid' / 'metadata_compiled.csv'
    if csv_path.exists():
        df = pd.read_csv(csv_path)
        # Map status to binary (COVID positive vs negative)
        return df.set_index('uuid')['status'].to_dict()
    return {}

def load_labels_coswara():
    """Load labels from Coswara metadata (COVID status)"""
    csv_path = DATASETS_ROOT / 'coswara' / 'combined_data.csv'
    if csv_path.exists():
        df = pd.read_csv(csv_path)
        return df.set_index('id')['covid_status'].to_dict()
    return {}

def load_labels_respiratory():
    """Load labels from Respiratory database (diagnosis)"""
    csv_path = DATASETS_ROOT / 'respiratory_sounds' / 'Respiratory_Sound_Database'
    for p in [csv_path / 'patient_diagnosis.csv', DATASETS_ROOT / 'respiratory_sounds' / 'patient_diagnosis.csv']:
        if p.exists():
            df = pd.read_csv(p)
            return df.set_index('Patient number')['Diagnosis'].to_dict()
    return {}

print("Label loading functions defined")

Label loading functions defined


In [4]:
class DiseaseClassifier:
    """Multi-head classifier for respiratory diseases"""
    
    def __init__(self, model_type='mlp'):
        self.model_type = model_type
        self.scaler = StandardScaler()
        self.label_encoder = LabelEncoder()
        self._init_model()
    
    def _init_model(self):
        if self.model_type == 'mlp':
            self.model = MLPClassifier(hidden_layer_sizes=(256, 128, 64), max_iter=500, early_stopping=True)
        elif self.model_type == 'rf':
            self.model = RandomForestClassifier(n_estimators=100, max_depth=10)
        elif self.model_type == 'gb':
            self.model = GradientBoostingClassifier(n_estimators=100, max_depth=5)
        else:
            self.model = LogisticRegression(max_iter=1000, C=0.1)
    
    def fit(self, X, y):
        X_scaled = self.scaler.fit_transform(X)
        y_encoded = self.label_encoder.fit_transform(y)
        self.model.fit(X_scaled, y_encoded)
        return self
    
    def predict(self, X):
        X_scaled = self.scaler.transform(X)
        y_pred = self.model.predict(X_scaled)
        return self.label_encoder.inverse_transform(y_pred)
    
    def predict_proba(self, X):
        X_scaled = self.scaler.transform(X)
        return self.model.predict_proba(X_scaled)
    
    def evaluate(self, X, y):
        y_pred = self.predict(X)
        y_encoded = self.label_encoder.transform(y)
        y_pred_encoded = self.label_encoder.transform(y_pred)
        
        f1 = f1_score(y_encoded, y_pred_encoded, average='weighted')
        try:
            y_proba = self.predict_proba(X)
            if len(self.label_encoder.classes_) == 2:
                auc = roc_auc_score(y_encoded, y_proba[:, 1])
            else:
                auc = roc_auc_score(y_encoded, y_proba, multi_class='ovr', average='weighted')
        except:
            auc = None
        
        return {'f1': f1, 'auc': auc, 'report': classification_report(y, y_pred)}

print("✓ DiseaseClassifier defined")

✓ DiseaseClassifier defined


In [5]:
def train_tb_classifier():
    """Train TB/COVID classifier using Coughvid + Coswara"""
    print("\n" + "="*50)
    print("TRAINING TB/COVID CLASSIFIER")
    print("="*50)
    
    # Combine embeddings from both datasets
    X_list, y_list = [], []
    
    for dataset in ['coughvid', 'coswara']:
        if dataset in embeddings:
            print(f"Using {dataset} embeddings: {embeddings[dataset].shape}")
            # For demo: create synthetic labels (0=healthy, 1=positive)
            n = len(embeddings[dataset])
            X_list.append(embeddings[dataset])
            y_list.append(np.random.randint(0, 2, n))  # Replace with actual labels
    
    if not X_list:
        print("No data available")
        return None
    
    X = np.vstack(X_list)
    y = np.concatenate(y_list)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
    
    clf = DiseaseClassifier('mlp')
    clf.fit(X_train, y_train)
    results = clf.evaluate(X_test, y_test)
    
    print(f"F1 Score: {results['f1']:.4f}")
    print(f"AUC-ROC: {results['auc']:.4f}" if results['auc'] else "AUC: N/A")
    
    return clf, results

tb_clf, tb_results = train_tb_classifier()


TRAINING TB/COVID CLASSIFIER
Using coughvid embeddings: (11319, 1024)
Using coswara embeddings: (5, 1024)
F1 Score: 0.5041
AUC-ROC: 0.5167


In [6]:
def train_parkinsons_classifier():
    """Train Parkinson's classifier"""
    print("\n" + "="*50)
    print("TRAINING PARKINSON'S CLASSIFIER")
    print("="*50)
    
    if 'parkinsons' not in embeddings:
        print("Parkinson's embeddings not available")
        return None, None
    
    X = embeddings['parkinsons']
    y = np.random.randint(0, 2, len(X))  # Replace with actual labels
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
    
    clf = DiseaseClassifier('mlp')
    clf.fit(X_train, y_train)
    results = clf.evaluate(X_test, y_test)
    
    print(f"F1 Score: {results['f1']:.4f}")
    print(f"AUC-ROC: {results['auc']:.4f}" if results['auc'] else "AUC: N/A")
    
    return clf, results

pd_clf, pd_results = train_parkinsons_classifier()


TRAINING PARKINSON'S CLASSIFIER


ValueError: The test_size = 1 should be greater or equal to the number of classes = 2

In [ ]:
def train_pulmonary_classifier():
    """Train Pulmonary anomaly classifier"""
    print("\n" + "="*50)
    print("TRAINING PULMONARY ANOMALY CLASSIFIER")
    print("="*50)
    
    if 'respiratory_sounds' not in embeddings:
        print("Respiratory sounds embeddings not available")
        return None, None
    
    X = embeddings['respiratory_sounds']
    # Labels: 0=normal, 1=crackles, 2=wheezes, 3=both
    y = np.random.randint(0, 4, len(X))  # Replace with actual labels
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
    
    clf = DiseaseClassifier('mlp')
    clf.fit(X_train, y_train)
    results = clf.evaluate(X_test, y_test)
    
    print(f"F1 Score: {results['f1']:.4f}")
    print(f"AUC-ROC: {results['auc']:.4f}" if results['auc'] else "AUC: N/A")
    
    return clf, results

pulm_clf, pulm_results = train_pulmonary_classifier()


TRAINING PULMONARY ANOMALY CLASSIFIER
F1 Score: 0.2589
AUC-ROC: 0.5020


In [ ]:
# Save models
import pickle

models = {'tb': tb_clf, 'parkinsons': pd_clf, 'pulmonary': pulm_clf}
for name, clf in models.items():
    if clf:
        with open(MODELS_DIR / f"{name}_classifier.pkl", 'wb') as f:
            pickle.dump(clf, f)
        print(f"✓ Saved {name} classifier")

# Save results summary
summary = {
    'tb': {'f1': tb_results['f1'], 'auc': tb_results['auc']} if tb_results else None,
    'parkinsons': {'f1': pd_results['f1'], 'auc': pd_results['auc']} if pd_results else None,
    'pulmonary': {'f1': pulm_results['f1'], 'auc': pulm_results['auc']} if pulm_results else None
}
with open(MODELS_DIR / 'training_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print(f"\nModels saved to: {MODELS_DIR}")
print("\nRun validation.py for cross-validation evaluation")

NameError: name 'pd_clf' is not defined